In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from crypto_trading_env import CryptoTradingEnv

2024-11-29 18:44:18.191772: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-29 18:44:18.193605: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-29 18:44:18.198646: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732916658.207055    1460 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732916658.209967    1460 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 18:44:18.219681: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
data = pd.read_parquet('../btc_hist_partitioned.parquet')
data = data.reset_index(drop=True).drop(columns=['Date'])
data.head()

,Open,High,Low,Close,m_avg_7,m_avg_25,m_avg_99,close_diff,m_avg_7_diff,m_avg_25_diff,m_avg_99_diff
0,3849.216309,3947.981201,3817.409424,3943.409424,3893.464722,3893.464722,3893.464722,0.025989,0.012995,0.012995,0.012995
1,3931.048584,3935.685059,3826.222900,3836.741211,3874.556885,3874.556885,3874.556885,-0.027050,-0.004856,-0.004856,-0.004856
2,3832.040039,3865.934570,3783.853760,3857.717529,3870.347046,3870.347046,3870.347046,0.005467,-0.001087,-0.001087,-0.001087
3,3851.973877,3904.903076,3836.900146,3845.194580,3865.316553,3865.316553,3865.316553,-0.003246,-0.001300,-0.001300,-0.001300
4,3836.519043,4093.297363,3826.513184,4076.632568,3900.535889,3900.535889,3900.535889,0.060189,0.009112,0.009112,0.009112


In [3]:
env = CryptoTradingEnv(
    dataset=data,
    initial_balance=1000,
    max_lose_percent=0.5
)

In [4]:
n_inputs = 11

model = keras.models.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=[n_inputs]), # try reku, elu, leaky relu
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(3, activation='softmax') # 3 actions: hold, buy, sell exclusively
])

/home/hugo/miniconda3/envs/postech/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1732916659.509793    1460 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 963 (3.76 KB)

 Trainable params: 963 (3.76 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# obs = env.reset()
obs, _, _  = env.step(2)
probas = model(obs.values[np.newaxis]) # predict the probability of going
print(probas, np.argmax(probas))

tf.Tensor([[1. 0. 0.]], shape=(1, 3), dtype=float32) 0


In [7]:
print(obs)

Open             3931.048584
High             3935.685059
Low              3826.222900
Close            3836.741211
m_avg_7          3874.556885
m_avg_25         3874.556885
m_avg_99         3874.556885
close_diff         -0.027050
m_avg_7_diff       -0.004856
m_avg_25_diff      -0.004856
m_avg_99_diff      -0.004856
Name: 1, dtype: float64


In [ ]:
def play_one_step(env, obs, model, loss_fn):
    with tf.GradientTape() as tape:
        ### Pós erro
        if np.isnan(obs).any() or np.isinf(obs).any():
            raise ValueError("obs contém valores NaN ou Inf")
        ###
        probs = model(obs.values[np.newaxis]) # predict the probability of going left
        if np.isnan(probs.numpy()).any():
            print(f"obs: {obs}")
            print(f"probs: {probs}")
            raise ValueError("probs contém valores NaN")
        action = np.random.choice(a=len(probs.numpy().flatten()),
                                   p=probs.numpy().flatten()) # random action based on the probability
        y_target = tf.constant([[2.]]) - tf.cast(action, tf.float32)
        print(f"y_target: {y_target}\nprobs: {probs}")
        loss = tf.reduce_mean(loss_fn(y_target[tf.newaxis], probs)) # loss function
        ### Pós erro
        if tf.math.is_nan(loss):
            raise ValueError("Loss gerou NaN")
        ###
    grads = tape.gradient(loss, model.trainable_variables) # compute the gradients
    obs, reward, done = env.step(action) # apply the action
    return obs, reward, done, grads

def play_multiple_episodes(env, n_episodes, n_max_steps, model, loss_fn):
    all_rewards = []
    all_grads = []
    max_steps = []
    for episode in range(n_episodes):
        current_rewards = []
        current_grads = []
        current_max_steps = 0
        obs = env.reset()
        for step in range(n_max_steps):
            obs, reward, done, grads = play_one_step(env, obs, model, loss_fn)
            current_rewards.append(reward)
            current_grads.append(grads)
            if done:
                if step > current_max_steps:
                    current_max_steps = step
                break
        else: # executed if the loop ended without break
            current_max_steps = n_max_steps
        max_steps.append(current_max_steps)
        all_rewards.append(current_rewards)
        all_grads.append(current_grads)
    return all_rewards, all_grads, max_steps

def discount_rewards(rewards, discount_rate):
    discounted = np.array(rewards)
    for step in range(len(rewards) - 2, -1, -1): # iterate over the rewards in reverse order
        discounted[step] += discounted[step + 1] * discount_rate # add the discounted reward from the next step
    return discounted

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate) for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean) / reward_std for discounted_rewards in all_discounted_rewards]

In [9]:
n_iterations = 150
n_episodes_per_update = 5
n_max_steps = len(data) - 1
discount_rate = 0.95

optimizer = keras.optimizers.Adam(learning_rate=0.01)
loss_fn = keras.losses.categorical_crossentropy

for iteration in range(n_iterations):
    all_rewards, all_grads, max_steps = play_multiple_episodes(env, n_episodes_per_update, n_max_steps, model, loss_fn)
    print(f"Iteration {iteration}:\n\tmean rewards = {np.mean([sum(rewards) for rewards in all_rewards])};\n\tmax steps = {np.max(max_steps)};\n\tmean steps = {np.mean(max_steps)}")
    all_final_rewards = discount_and_normalize_rewards(all_rewards, discount_rate)
    all_mean_grads = []
    for var_index in range(len(model.trainable_variables)): # iterate over all trainable variables
        mean_grads = tf.reduce_mean([final_reward * all_grads[episode_index][step][var_index] # mean gradient for this variable
                                     for episode_index, final_rewards in enumerate(all_final_rewards)
                                     for step, final_reward in enumerate(final_rewards)], axis=0)
        all_mean_grads.append(mean_grads)
    optimizer.apply_gradients(zip(all_mean_grads, model.trainable_variables)) # apply the gradients

ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(1, 1, 1), output.shape=(1, 3)